In [73]:
!pip install torch torchvision

In [74]:
!python -m pip install torch

In [0]:
# Dependencies
import pandas as pd 
import numpy as np 
import torch.nn as nn

import torch
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report

import warnings
warnings.filterwarnings("ignore")

In [76]:
# Read in data from S3 Buckets

url = "https://job-postings-dataviz.s3.amazonaws.com/fake_jobs_clean.csv"

df = pd.read_csv(url, sep=",",  encoding = "UTF-8" )

# Show DataFrame
df.head()

,job_id,city,state/province,country,title,department,industry,function,salary_range,salary_provided,company_profile,description,requirements,benefits,benefits_provided,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,fraudulent
0,1,New York,NY,US,Marketing Intern,Marketing,NaN,Marketing,NaN,0,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,0,1,0,Other,Internship,NaN,0
1,2,Auckland,,NZ,Customer Service - Cloud Video Production,Success,Marketing and Advertising,Customer Service,NaN,0,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,1,0,1,0,Full-time,Not Applicable,NaN,0
2,3,Wever,IA,US,Commissioning Machinery Assistant (CMA),NaN,NaN,NaN,NaN,0,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,0,1,0,NaN,NaN,NaN,0
3,4,Washington,DC,US,Account Executive - Washington DC,Sales,Computer Software,Sales,NaN,0,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,1,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,0
4,5,Fort Worth,FL,US,Bill Review Manager,NaN,Hospital & Health Care,Health Care Provider,NaN,0,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,1,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,0


In [77]:
# Focus only on the job description. Create a dataFrame with label "fraudulent" and description
descrip_df =df [['fraudulent','description']]
descrip_df.head()


,fraudulent,description
0,0,"Food52, a fast-growing, James Beard Award-winn..."
1,0,Organised - Focused - Vibrant - Awesome!Do you...
2,0,"Our client, located in Houston, is actively se..."
3,0,THE COMPANY: ESRI – Environmental Systems Rese...
4,0,JOB TITLE: Itemization Review ManagerLOCATION:...


In [78]:
# Drop NaN values but dropping duplicates is qustionable
# these duplicates are real data and contribute to both classes
print(len(descrip_df))
descrip_df= descrip_df.dropna()
print(len(descrip_df))
# descrip_df = descrip_df.drop_duplicates()
# print(descrip_df.count())

17880
17879


In [80]:
# Make sure number of real vs fraudulent are more balanced
df_fake= descrip_df[descrip_df['fraudulent'] == 1] 
df_real = descrip_df[descrip_df['fraudulent'] == 0] 
# Count how many Real vs Fraudulent postings
n_f=len(df_fake)
n_n=len(df_real)
print(n_f)
print(n_n)


865
17014


Create a data set with same instances of fake and real :UNDERSTIMATING the real posts

In [82]:
df_new= df_real.sample  ( n_f  , random_state=580)
df_unders = df_new.append(df_fake)
print(len(df_new))
print(len(df_unders))

865
1730


In [84]:
# Shuffle the dataframe so fraudulent postings more evenly distributed 
df_underst = df_unders.sample(len(df_unders), random_state=580) 
# df_underst.head(20)
X= df_underst.iloc[:,1].values
y=df_underst.iloc[:,0].values

X

array(['Exciting opportunity to work with fast paced company that provides church management software and web solutions!\xa0 We are seeking a highly motivated, exceptional Web Developer.\xa0 Company offers a business casual environment and a great Ann Arbor location.\xa0Job Summary: This position is for a developer who will be responsible for the future releases of the ParishSOFT suite of applications, maintaining existing applications, and also providing technical assistance to other departments. This position will work with project managers to ensure that project deadlines are met and that the work meets user requirements.',
       'Greetings from VAM SYSTEMS…..VAM SYSTEMS\xa0is a Business Consulting, IT Solutions and Services\xa0company with operations in UAE, Qatar, Bahrain, USA, Australia, Singapore &amp; India. We offers a comprehensive list of services in the field of IT infrastructure management, Cloud services, IT Consulting Services, Banking, Management Resources, Information

### Feature Transformations (Term Frequency times inverse document frequency)


In [85]:
# feature extraction using "bag of words" model. Create feature vectors. 
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))


count_vectorizer = CountVectorizer( stop_words=stopwords.words('english'))
tfidfconverter = TfidfTransformer()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
X_c = count_vectorizer.fit_transform(X)
X_tfidf = tfidfconverter.fit_transform(X_c).toarray()

y.shape

Create a Naive Bayes Model  (ref : https://scikit-learn.org/stable/modules/naive_bayes.html)

In [0]:
# Break the data in trainand test datasets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, random_state=580, stratify=y)

Multinomial Naive Bayes

In [0]:
# Create a Naive Bayes model and fit training data

from sklearn.naive_bayes import MultinomialNB

m_nb = MultinomialNB()
predictor = m_nb.fit(X_train,y_train)

In [0]:
# Tranform the model with the testing data
test_results = predictor.predict(X_test)

In [95]:
from sklearn.metrics import classification_report

print(classification_report(y_test, test_results,
                            target_names= ["real", "fake"]))

              precision    recall  f1-score   support

        real       0.86      0.80      0.83       217
        fake       0.81      0.88      0.84       216

    accuracy                           0.84       433
   macro avg       0.84      0.84      0.84       433
weighted avg       0.84      0.84      0.84       433



Save the model 

In [96]:
# save the model
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [100]:
# Save to the model to the local drive

from sklearn.externals import joblib

filename = '/content/drive/My Drive/NB_under.h5'
joblib.dump(predictor, filename)

['/content/drive/My Drive/NB_under.h5']

In [0]:
# Load the model
loaded_model= joblib.load(filename)
predictionsl = loaded_model.predict(X_test)


In [105]:
from sklearn.metrics import classification_report

print(classification_report(y_test, test_results,
                            target_names= ["real", "fake"]))

              precision    recall  f1-score   support

        real       0.86      0.80      0.83       217
        fake       0.81      0.88      0.84       216

    accuracy                           0.84       433
   macro avg       0.84      0.84      0.84       433
weighted avg       0.84      0.84      0.84       433

